In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Import Data

In [3]:
df_fhvhv = pd.read_parquet('../fhvhv_tripdata_2019-02.parquet', engine='pyarrow')
df_fhvhv.head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0003,B02867,B02867,2019-02-01 00:01:26,2019-02-01 00:02:55,2019-02-01 00:05:18,2019-02-01 00:14:57,245,251,2.45,...,0.83,0.0,None,0.0,7.48,Y,N,N,N,None
1,HV0003,B02879,B02879,2019-02-01 00:26:08,2019-02-01 00:41:29,2019-02-01 00:41:29,2019-02-01 00:49:39,216,197,1.71,...,0.70,0.0,None,2.0,7.93,N,N,N,N,None
2,HV0005,B02510,None,2019-02-01 00:48:58,NaT,2019-02-01 00:51:34,2019-02-01 01:28:29,261,234,5.01,...,3.99,0.0,None,0.0,35.97,N,Y,N,N,None
3,HV0005,B02510,None,2019-02-01 00:02:15,NaT,2019-02-01 00:03:51,2019-02-01 00:07:16,87,87,0.34,...,0.64,0.0,None,3.0,5.39,N,Y,N,N,None
4,HV0005,B02510,None,2019-02-01 00:06:17,NaT,2019-02-01 00:09:44,2019-02-01 00:39:56,87,198,6.84,...,2.16,0.0,None,4.0,17.07,N,Y,N,N,None


In [6]:
df_fhvhv.shape

(20159102, 24)

共有24個columns，分兩部分查看column info

In [4]:
df_fhvhv.iloc[:, :11].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20159102 entries, 0 to 20159101
Data columns (total 11 columns):
 #   Column                Dtype         
---  ------                -----         
 0   hvfhs_license_num     object        
 1   dispatching_base_num  object        
 2   originating_base_num  object        
 3   request_datetime      datetime64[ns]
 4   on_scene_datetime     datetime64[ns]
 5   pickup_datetime       datetime64[ns]
 6   dropoff_datetime      datetime64[ns]
 7   PULocationID          int64         
 8   DOLocationID          int64         
 9   trip_miles            float64       
 10  trip_time             int64         
dtypes: datetime64[ns](4), float64(1), int64(3), object(3)
memory usage: 1.7+ GB


In [5]:
df_fhvhv.iloc[:, 12:].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20159102 entries, 0 to 20159101
Data columns (total 12 columns):
 #   Column                Dtype  
---  ------                -----  
 0   tolls                 float64
 1   bcf                   float64
 2   sales_tax             float64
 3   congestion_surcharge  float64
 4   airport_fee           object 
 5   tips                  float64
 6   driver_pay            float64
 7   shared_request_flag   object 
 8   shared_match_flag     object 
 9   access_a_ride_flag    object 
 10  wav_request_flag      object 
 11  wav_match_flag        object 
dtypes: float64(6), object(6)
memory usage: 1.8+ GB


原始資料有兩千多萬筆，避免處理時間過長，僅擷取2/1資料

In [8]:
df_0201 = df_fhvhv[df_fhvhv['pickup_datetime'].dt.day == 1]
df_0201.shape

(860161, 24)

## EDA

查看每種license_num各有多少record

In [9]:
df_0201.hvfhs_license_num.value_counts(normalize=True)

HV0003    0.664504
HV0005    0.229442
HV0002    0.055554
HV0004    0.050501
Name: hvfhs_license_num, dtype: float64

由於Juno(HV0002)和Via(HV0004)資料量僅占10%左右，與Uber(HV0003)、Lyft(HV0005)有相當大的差距，僅保留Uber和Lyft的資料

In [34]:
df_0201 = df_0201[df_0201['hvfhs_license_num'].isin(['HV0003', 'HV0005'])]
df_0201.hvfhs_license_num.value_counts()

HV0003    571580
HV0005    197357
Name: hvfhs_license_num, dtype: int64

In [16]:
df_0201.total_amount = df_0201[['base_passenger_fare', 'tolls', 'bcf', 'sales_tax', 'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay']].sum(axis=1)
df_0201.total_amount

C:\Users\Ticy\AppData\Local\Temp\ipykernel_4616\367784853.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_10000.total_amount = df_10000[['base_passenger_fare', 'tolls', 'bcf', 'sales_tax', 'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay']].sum(axis=1)


0       17.89
1       18.74
2       86.04
3       16.40
4       48.20
        ...  
9995    17.12
9996     6.80
9997     7.98
9998    38.77
9999    10.38
Length: 10000, dtype: float64

In [19]:
df_0201.cal_tax_perc = df_0201.sales_tax / (df_0201.total_amount - df_0201.sales_tax)
df_0201.cal_tax_perc

0       0.048652
1       0.038803
2       0.048629
3       0.040609
4       0.046916
          ...   
9995    0.086294
9996    0.086262
9997    0.087193
9998    0.051247
9999    0.036963
Length: 10000, dtype: float64